In [1]:
%run -i turing.py
init()

:0: FutureWarning: IPython widgets are experimental and may change in the future.


# Computation of Turing machines

In [2]:
def run(transitions, input, steps):
    """simulate Turing machine for the given number of steps and the given input"""

    # convert input from string to list of symbols
    # we use '|>' as a symbol to indicate the beginning of the tape
    input = ['|>'] + list(input) + [' ']

    # sanitize transitions for 'accept' and 'reject' states and for symbol '|>'
    transitions = sanitize_transitions(transitions)

    # create initial configuration
    c = Configuration(state='start', head=1, tape=input)

    for i in xrange(0, steps):
        # read tape content under head
        state = c.state
        read = c.tape[c.head]

        # lookup transition based on state and read symbol
        next_state, write, head_delta = transitions(state, read)

        # update configuration
        c.state = next_state
        c.tape[c.head] = write
        c.head += head_delta
        if c.head >= len(c.tape):
            c.tape += [' ']

    # return final configuration
    return c


In [3]:
def check_transitions(transitions, states, alphabet):

    transitions = sanitize_transitions(transitions)

    for state in states:
        for read in alphabet:
            next_state, write, head_delta = transitions(state, read)

            # we either move one position to the left or to the right
            assert(head_delta in [-1,1])

            # if we read the begin symbol,
            if read == '|>':
                # we need to write it back
                assert(write == '|>')
                # we need to move to the right
                assert(head_delta == 1)
            else:
                # we cannot write the begin symbol
                assert(write != '|>')

            # if we are in one of the final states
            if state in ['accept', 'reject', 'halt']:
                # we cannot change to a different state
                assert(next_state == state)

    print("transition checks passed")


# Examples
## Copy machine

In [4]:
def transitions_copy(state, read):
    if state == 'start':
        if 'write' not in read:
            return read + '-write', read + '-read', 1
        else:
            return 'accept', read, 1
    elif 'write' in state:
        if read != ' ':
            return state, read, 1
        else:
            return 'rewind', state, -1
    elif state == 'rewind':
        if 'read' not in read:
            return state, read, -1
        else:
            return 'start', read, 1

In [18]:
simulate({'copy': transitions_copy}, transitions_default=transitions_copy, unary=False)

<__main__.Configuration instance at 0x7f1e71a20d88>

In [6]:
transitions_table(transitions_copy, 
                  ['start', '0-write', '1-write', 'rewind'],
                  ['0', '1', '0-read', '1-read', '0-write', '1-write'])

transition checks passed


,state,read,next_state,write,head_delta
0,start,0,0-write,0-read,1
1,start,1,1-write,1-read,1
2,start,0-read,0-read-write,0-read-read,1
3,start,1-read,1-read-write,1-read-read,1
4,start,0-write,accept,0-write,1
5,start,1-write,accept,1-write,1
6,0-write,0,0-write,0,1
7,0-write,1,0-write,1,1
8,0-write,0-read,0-write,0-read,1
9,0-write,1-read,0-write,1-read,1


## Power-of-2 machine

In [7]:
def transitions_power(state,read):
    if state == 'rewind':
        return state, read, -1
    elif read == 'x':
        return state, read, 1  
    elif state == 'start':
        if read != '1':
            return 'reject', read, 1
        else: 
            return 'start-even', read, 1
    elif 'even' in state and read == '1':
        return 'odd', 'x', 1
    elif state=='odd' and read == '1':
        return 'even', read, 1
    elif state=='odd':
        if read == ' ':
            return 'rewind', '2', -1
        else:
            return state, read, 1
    elif state=='start-even' and read != '1':
        return 'accept', read, -1
    elif state=='even' and read != '1':
        return 'reject', read, -1
        

In [16]:
simulate({'power': transitions_power}, transitions_default=transitions_power, unary=True)

<__main__.Configuration instance at 0x7f1e71aec248>

In [9]:
transitions_table(transitions_power, 
                  ['start', 'start-even', 'even', 'odd', 'rewind'], 
                  ['0', '1', 'x', ' ', '|>'])

transition checks passed


,state,read,next_state,write,head_delta
0,start,0,reject,0,1
1,start,1,start-even,1,1
2,start,x,start,x,1
3,start,,reject,,1
4,start,|>,start,|>,1
5,start-even,0,accept,0,-1
6,start-even,1,odd,x,1
7,start-even,x,start-even,x,1
8,start-even,,accept,,-1
9,start-even,|>,start,|>,1
